In [1]:
import pandas as pd
from scipy import stats
import numpy as np
from pandas.api.types import CategoricalDtype
import re
from helpers import get_renamed_blocks, get_renamed_blocks3

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
prtr = pd.read_excel("../data/2022-12-15_PRTR-Deutschland_Freisetzungen - Excel.xlsx")

In [4]:
#STATES = ['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt', 'Kohlestromvermarktungsverbot']
STATES = ['in Betrieb', 'Kapazitätsreserve', 'Netzreserve', 'Strommarktrückkehr', 'vorläufig stillgelegt', 'stillgelegt §13b EnWG', 'stillgelegt KVBG', 'stillgelegt']
ACTIVE_STATES = ['in Betrieb', 'Kapazitätsreserve', 'Netzreserve', 'Strommarktrückkehr', 'vorläufig stillgelegt']
ENERGIES = ["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte", "Abfall", "Biomasse", ""]

In [5]:
plr = pd.read_excel("../data/Kraftwerksliste_2022.xlsx", sheet_name=1, skiprows=9)

In [6]:
pl0 = pd.read_excel("../data/Kraftwerksliste_2022.xlsx", skiprows=10)

In [7]:
#list(plr)

In [8]:
#list(pl0_bak)

In [9]:
pl0_bak = pl0.copy()
plr_bak = plr.copy()

In [10]:
#list(pl0_bak)

In [11]:
cols = [11,12,14,16,17,18,20,22,23,24,25]
plt = get_renamed_blocks(pl0)
plq = plt.drop(plt.columns[cols],axis=1)

In [12]:
cols2 = [11,12,14,16,17,20,21,22]
plr0a = get_renamed_blocks3(plr)
plr0b = plr0a.drop(plr0a.columns[cols2],axis=1)

In [13]:
plr0b

,blockid,bnaid,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
0,BNA0302,BNA0302,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,1957,2011,Endgültig Stillgelegt 2011 (ohne StA),Braunkohle,Nein,129.000,NaN
1,BNA0303,BNA0303,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,1957,2011,Endgültig Stillgelegt 2011 (ohne StA),Braunkohle,Nein,130.000,NaN
2,BNA0304,BNA0304,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,1960,2011,Endgültig Stillgelegt 2011 (ohne StA),Braunkohle,Nein,124.000,NaN
3,BNA0414,BNA0414,Westfalen,59071.0,Hamm-Uentrop,NaN,NaN,Nordrhein-Westfalen,1963,2011,Endgültig Stillgelegt 2011 (ohne StA),Steinkohle,Nein,152.000,NaN
4,BNA0415,BNA0415,Westfalen,59071.0,Hamm-Uentrop,NaN,NaN,Nordrhein-Westfalen,1963,2011,Endgültig Stillgelegt 2011 (ohne StA),Steinkohle,Nein,152.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,SEE962318991879,NaN,Kraftwerk Töging MS5,84513.0,Töging,Werkstraße,18,Bayern,1924-01-01 00:00:00,2022-02-01 00:00:00,endgültig stillgelegt 2022 (ohne § 13b EnWG od...,Wasser,Nein,6.093,NaN
186,SEE965312155841,NaN,Kraftwerk Töging MS6,84513.0,Töging,Werkstraße,18,Bayern,1924-01-01 00:00:00,2022-02-01 00:00:00,endgültig stillgelegt 2022 (ohne § 13b EnWG od...,Wasser,Nein,6.093,NaN
187,SEE942159239255,NaN,Kraftwerk Töging MS7,84513.0,Töging,Werkstraße,18,Bayern,1924-01-01 00:00:00,2022-02-01 00:00:00,endgültig stillgelegt 2022 (ohne § 13b EnWG od...,Wasser,Nein,6.093,NaN
188,SEE959664119668,NaN,Kraftwerk Töging MS8,84513.0,Töging,Werkstraße,18,Bayern,1924-01-01 00:00:00,2022-02-01 00:00:00,endgültig stillgelegt 2022 (ohne § 13b EnWG od...,Wasser,Nein,6.093,NaN


In [14]:
plq

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,state,energysource,chp,power,tech
0,SEE946736241334,NaN,Palm Power GmbH & Co. KG,HKW Aalen,73432,Aalen,Palm Allee,1,Baden-Württemberg,2021-11-11,In Betrieb,Erdgas,Ja,77.000000,Gasturbinen mit nachgeschalteter Dampfturbine
1,SEE930596800480,BNA0005,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT A,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,1990-12-22,Kapazitätsreserve aufgrund von § 13e EnWG,Erdgas,Nein,37.104000,Gasturbinen mit Abhitzekessel
2,SEE929797382345,BNA0006,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT B,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,1990-12-10,Kapazitätsreserve aufgrund von § 13e EnWG,Erdgas,Nein,37.104000,Gasturbinen mit Abhitzekessel
3,SEE988046628214,BNA0007,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT C,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,1991-03-26,Kapazitätsreserve aufgrund von § 13e EnWG,Erdgas,Nein,37.104000,Gasturbinen mit Abhitzekessel
4,SEE923527691592,BNA0008,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT D,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,1992-02-20,Kapazitätsreserve aufgrund von § 13e EnWG,Erdgas,Nein,37.104000,Gasturbinen mit Abhitzekessel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940,SEE9-Dummy-nicht_EEG,NaN,Nicht-EEG-Anlagen < 10 MW,NaN,NaN,NaN,NaN,NaN,ohne Zuordnung,NaT,In Betrieb,Sonstige Energieträger (nicht erneuerbar),NaN,582.867492,NaN
1941,SEE9-Dummy-nicht_EEG,NaN,Nicht-EEG-Anlagen < 10 MW,NaN,NaN,NaN,NaN,NaN,ohne Zuordnung,NaT,In Betrieb,Batteriespeicher,NaN,326.863412,NaN
1942,SEE9-Dummy-nicht_EEG,NaN,Nicht-EEG-Anlagen < 10 MW,NaN,NaN,NaN,NaN,NaN,ohne Zuordnung,NaT,In Betrieb,Steinkohle,NaN,49.943000,NaN
1943,SEE9-Dummy-nicht_EEG,NaN,Nicht-EEG-Anlagen < 10 MW,NaN,NaN,NaN,NaN,NaN,ohne Zuordnung,NaT,In Betrieb,Sonstige Energieträger (nicht erneuerbar),NaN,0.143500,NaN


In [15]:
plq0a = plq[~plq['blockid'].isin(["SEE9-Dummy-nicht_EEG", "SEE9-Dummy-EEG"])]

In [16]:
plq0b = plq0a.sort_values(['power'], ascending=False)

In [17]:
#plq0b.groupby("energysource").count()

In [18]:
#plq0b.loc[pl0.energysource == "Abfall"]

In [19]:
#pl0 = pd.read_csv("../Kraftwerksliste_2020.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
#prtr = pd.read_csv("../data/2019-11-25_PRTR-Deutschland_Freisetzungen.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=0)
#pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python")
bm = pd.read_csv("plant_sse_mapper.csv", engine="python")

In [20]:
#bm

In [21]:
bm['plantid'] = bm['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [22]:
bm.loc[bm.plantid.str.contains("_")]

,plantid,sseid
114,06-04-11_2001178_8_0,BNA0140
115,06-04-11_2001178_8_0,SEE968950464368
116,06-04-11_2001178_8_1,SEE958310939391
117,06-04-11_2001178_8_1,BNA0146
118,06-04-11_2001178_8_1,BNA0145
119,06-04-11_2001178_8_2,BNA0143
120,06-04-11_2001178_8_2,SEE989202977269
121,06-04-11_2001178_8_2,SEE924172977710
122,06-04-11_2039669_2_0,SEE904968236689


In [23]:
#prtr.describe()

In [24]:
plq0a.insert(10, "endop", np.nan)
plr0b.insert(2, "company", np.nan)

In [25]:
plq0a.shape

(1783, 16)

In [26]:
plr0b.shape

(190, 16)

In [27]:
pl_combined = pd.concat([plq0a, plr0b])

In [28]:
pl_combined['energysource'] = pl_combined['energysource'].apply(lambda x: str(x).replace('Wärme', 'Erdgas'))

In [736]:
pl_combined.loc[pl_combined.blockid == "BNA0711"]

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
8,BNA0711,BNA0711,NaN,Niederaußem,50129.0,Bergheim,NaN,NaN,Nordrhein-Westfalen,1963,2012,Endgültig Stillgelegt 2012 (ohne StA),Braunkohle,Nein,125.0,NaN


In [30]:
pl_combined.shape

(1973, 16)

In [199]:
#list(plq0a)

In [200]:
#list(plr0b)

In [201]:
#list(pl_combined)

In [34]:
#list(plr0b)

In [35]:
#list(plq0a)

In [36]:
pl0 = pl_combined.copy()

In [37]:
pl0.loc[pl0["blockid"] == "BNA0645"]

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech


In [38]:
#pl0.groupby("energysource").count()

In [39]:
#pl0.groupby("state").count()

In [40]:
pl1 = pl0.loc[pl0["energysource"].isin(ENERGIES)]

In [41]:
#pl0.loc[pl0.energysource == "Wärme"]

In [737]:
pl_debug = pl0.copy()
pl_debug['initialop'] = pd.to_datetime(pl_debug['initialop'], errors='coerce')
pl_debug2 = pl_debug.copy()

In [738]:
pl_debug2.loc[pl_debug2.initialop == pd.NaT ]

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech


In [739]:
pl_debug2['initialop'] = pd.DatetimeIndex(pl_debug2.initialop).year
#pl_debug2['initialop'] = pl_debug.initialop.dt.year

In [45]:
#pl_debug2['initialop'] = pl_debug.initialop.dt.year

In [46]:
#pl_debug['initialop'] = pl_debug['initialop'].to_datetime()

In [47]:
pl_debug2.groupby("initialop").count()

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,endop,state,energysource,chp,power,tech
initialop,,,,,,,,,,,,,,,
1900.0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1
1905.0,2,2,2,2,2,2,2,0,2,0,2,2,2,2,0
1912.0,5,5,5,5,5,5,5,5,5,0,5,5,5,5,0
1915.0,2,2,2,2,2,2,2,0,2,0,2,2,2,2,0
1918.0,7,5,7,7,7,7,7,7,7,0,7,7,7,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018.0,59,42,59,59,59,59,57,54,59,0,59,59,59,59,50
2019.0,27,23,27,27,27,27,27,27,27,0,27,27,27,27,22
2020.0,30,20,30,30,30,30,29,28,30,0,30,30,30,30,27


In [48]:
#pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [49]:
#pl0

In [50]:
#bm

In [51]:
#pl0.groupby("Energieträger").count()

In [52]:
#list(plt)

In [53]:
pl1

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
0,SEE946736241334,NaN,Palm Power GmbH & Co. KG,HKW Aalen,73432,Aalen,Palm Allee,1,Baden-Württemberg,2021-11-11 00:00:00,NaN,In Betrieb,Erdgas,Ja,77.000,Gasturbinen mit nachgeschalteter Dampfturbine
1,SEE930596800480,BNA0005,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT A,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,1990-12-22 00:00:00,NaN,Kapazitätsreserve aufgrund von § 13e EnWG,Erdgas,Nein,37.104,Gasturbinen mit Abhitzekessel
2,SEE929797382345,BNA0006,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT B,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,1990-12-10 00:00:00,NaN,Kapazitätsreserve aufgrund von § 13e EnWG,Erdgas,Nein,37.104,Gasturbinen mit Abhitzekessel
3,SEE988046628214,BNA0007,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT C,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,1991-03-26 00:00:00,NaN,Kapazitätsreserve aufgrund von § 13e EnWG,Erdgas,Nein,37.104,Gasturbinen mit Abhitzekessel
4,SEE923527691592,BNA0008,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT D,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,1992-02-20 00:00:00,NaN,Kapazitätsreserve aufgrund von § 13e EnWG,Erdgas,Nein,37.104,Gasturbinen mit Abhitzekessel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,SEE963177319026,BNA0800,NaN,Turbine 4,75175.0,Pforzheim,Hohwiesenweg,15,Baden-Württemberg,1980-11-14 00:00:00,2022-05-18 00:00:00,endgültig stillgelegt 2022 (ohne § 13b EnWG od...,Erdgas,Ja,16.000,Kondensationsmaschine mit Entnahme
172,SEE974753539035,BNA0801,NaN,Turbine 6,75175.0,Pforzheim,Hohwiesenweg,15,Baden-Württemberg,1990-01-18 00:00:00,2022-05-18 00:00:00,endgültig stillgelegt 2022 (ohne § 13b EnWG od...,Steinkohle,Ja,26.900,Kondensationsmaschine mit Entnahme
173,SEE943101874623,BNA1264,NaN,GT21,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen,1995-12-02 00:00:00,2022-03-31 00:00:00,endgültig stillgelegt 2022 (ohne § 13b EnWG od...,Erdgas,Ja,4.400,Gasturbinen mit Abhitzekessel
174,SEE963023008050,BNA1264,NaN,GT22,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen,1995-12-02 00:00:00,2022-03-31 00:00:00,endgültig stillgelegt 2022 (ohne § 13b EnWG od...,Erdgas,Ja,4.400,Gasturbinen mit Abhitzekessel


In [54]:
pl1.loc[pl1.blockid == "BNA0221c"]

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
108,BNA0221c,BNA0221c,NaN,Gasblock,40221.0,Düsseldorf,Auf der Lausward,75,Nordrhein-Westfalen,1977-03-28 00:00:00,2019,Endgültig Stillgelegt 2019 (mit StA),Erdgas,Ja,293.0,NaN


In [55]:
plt = pl1.dropna(subset=["blockid"])

In [56]:
plt = plt.astype({"energysource": 'category'})

In [210]:
plt.loc[plt.blockid == "BNA0711"]

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
8,BNA0711,BNA0711,NaN,Niederaußem,50129.0,Bergheim,NaN,NaN,Nordrhein-Westfalen,1963,2012,Endgültig Stillgelegt 2012 (ohne StA),Braunkohle,Nein,125.0,NaN


In [57]:
# remove duplicates blocks in berlin, TODO: remove, because BNetzA switched to SEE ids no longer neccessary
#idxs = plt.index[plt["blockid"].isin(["BNA0083", "BNA0085b"]) & ~plt["state"].isin(["in Betrieb"])].tolist()
#plt2 = plt.loc[~plt.index.isin(idxs)]

In [58]:
#plt.loc[plt.index.isin(idxs)]#

In [59]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')
#plt["Energieträger"] = plt["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)

In [60]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')

In [61]:
#plt.groupby('Energieträger').count()

In [219]:
pl2 = plt.copy()

In [220]:
#pl2.groupby("Energieträger").count()

In [221]:
# plt.dtypes

In [222]:
#pl2 = plt.rename(columns={ pl1.columns[0]: "blockid", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [223]:
'''
def to_year(x):
    if type(x) == type(4):
        return x
    elif type(x) == type(2.5):
        return 0 # handle nans
    return int(x[-4:])
'''

'\ndef to_year(x):\n    if type(x) == type(4):\n        return x\n    elif type(x) == type(2.5):\n        return 0 # handle nans\n    return int(x[-4:])\n'

In [224]:
def to_year(x):
    dt = pd.to_datetime(x)
    return dt.year

In [225]:
#blocks.loc[blocks.Energieträger == "Kernenergie"]

In [226]:
#pl2

In [227]:
'''
pl2.loc[pl2.blockid == 'BNA0861a', 'initialop'] = 2012
pl2.loc[pl2.blockid == 'BNA1334', 'initialop'] = 2002
pl2.loc[pl2.blockid == 'BNA1141', 'initialop'] = 1970
pl2.loc[pl2.blockid == 'BNA0418', 'initialop'] = 2013
pl2.loc[pl2.blockid == 'BNA1499', 'initialop'] = 1951
pl2.loc[pl2.blockid == 'BNA1502', 'initialop'] = 2013
pl2.loc[pl2.blockid == 'BNA1500', 'initialop'] = 1990
pl2.loc[pl2.blockid == 'BNA1498', 'initialop'] = 1953
pl2.loc[pl2.blockid == 'BNA1260', 'initialop'] = 2013
pl2.loc[pl2.blockid == 'BNA1056', 'initialop'] = 2006
pl2.loc[pl2.blockid == 'BNA1114', 'initialop'] = 2012



pl2.loc[pl2.blockid == 'BNA0413b', 'initialop'] = 2014 # Westfalen D

pl2.loc[pl2.blockid == 'BNA0355', 'initialop'] = 1981 # Grafenrheinfeld
'''

#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

"\npl2.loc[pl2.blockid == 'BNA0861a', 'initialop'] = 2012\npl2.loc[pl2.blockid == 'BNA1334', 'initialop'] = 2002\npl2.loc[pl2.blockid == 'BNA1141', 'initialop'] = 1970\npl2.loc[pl2.blockid == 'BNA0418', 'initialop'] = 2013\npl2.loc[pl2.blockid == 'BNA1499', 'initialop'] = 1951\npl2.loc[pl2.blockid == 'BNA1502', 'initialop'] = 2013\npl2.loc[pl2.blockid == 'BNA1500', 'initialop'] = 1990\npl2.loc[pl2.blockid == 'BNA1498', 'initialop'] = 1953\npl2.loc[pl2.blockid == 'BNA1260', 'initialop'] = 2013\npl2.loc[pl2.blockid == 'BNA1056', 'initialop'] = 2006\npl2.loc[pl2.blockid == 'BNA1114', 'initialop'] = 2012\n\n\n\npl2.loc[pl2.blockid == 'BNA0413b', 'initialop'] = 2014 # Westfalen D\n\npl2.loc[pl2.blockid == 'BNA0355', 'initialop'] = 1981 # Grafenrheinfeld\n"

In [228]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['initialop'] = pl2['initialop'].apply(to_year)
#pl2['endop'] = pl2['endop'].apply(to_year)
pl2['power'] = pl2['power'].apply(pd.to_numeric)

In [229]:
pl2.loc[pl2.blockid == "BNA0711"]

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
8,BNA0711,BNA0711,NaN,Niederaußem,50129.0,Bergheim,NaN,NaN,Nordrhein-Westfalen,1970.0,2012,Endgültig Stillgelegt 2012 (ohne StA),Braunkohle,Nein,125.0,NaN


In [230]:
#bm[pl2.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

In [231]:
#pl2[pl2.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

In [232]:
#cols = [10,11,12,14,17,18,19]
#pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [233]:
#pl3.groupby('Unternehmen').max()

In [234]:
#pl3

In [235]:
def fix_company(company):
    #print(company)
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG", "Lausitz Energie": "LEAG"}
    for key, value in company_dict.items():
        if key in str(company):
            return value
    return company

In [236]:
def extract_still(x):
    match = re.findall("[0-9]{4}",x)
    return match[0] if match else np.nan

In [237]:
def fix_kwk(x):
    return "Nein" if x == "nein" else "Ja" if x == "ja" else x

In [238]:
pl4 = pl2.copy()

In [239]:
sq = 'Endgültig Stillgelegt 2013 (mit StA)'

In [240]:
list(pl2.groupby("state").count().reset_index()['state'])

['Endgültig Stillgelegt 2011 (ohne StA)',
 'Endgültig Stillgelegt 2012 (ohne StA)',
 'Endgültig Stillgelegt 2013 (mit StA)',
 'Endgültig Stillgelegt 2013 (ohne StA)',
 'Endgültig Stillgelegt 2014 (mit StA)',
 'Endgültig Stillgelegt 2014 (ohne StA)',
 'Endgültig Stillgelegt 2015 (mit StA)',
 'Endgültig Stillgelegt 2015 (ohne StA)',
 'Endgültig Stillgelegt 2016 (mit StA)',
 'Endgültig Stillgelegt 2016 (ohne StA)',
 'Endgültig Stillgelegt 2017 (mit StA)',
 'Endgültig Stillgelegt 2017 (ohne StA)',
 'Endgültig Stillgelegt 2018 (mit StA)',
 'Endgültig Stillgelegt 2018 (ohne StA)',
 'Endgültig Stillgelegt 2019 (mit StA)',
 'Endgültig Stillgelegt 2019 (ohne StA)',
 'In Betrieb',
 'Kapazitätsreserve aufgrund von § 13e EnWG',
 'Netzreserve aufgrund § 13b EnWG',
 'befristete Strommarktrückkehr',
 'endgültig stillgelegt 2017 (ohne § 13b EnWG oder KVBG)',
 'endgültig stillgelegt 2018 (ohne § 13b EnWG oder KVBG)',
 'endgültig stillgelegt 2020 (nach KVBG)',
 'endgültig stillgelegt 2020 (nach § 13b En

In [241]:
z = "endgültig stillgelegt 2022 (ohne § 13b EnWG oder KVBG)"

In [242]:
def extract_state(state):
    if state == "In Betrieb":
        return "in Betrieb"
    elif "Endgültig Stillgelegt" in state:
        return "stillgelegt"
    elif "Vorläufig Stillgelegt" in state:
        return "vorläufig stillgelegt"
    elif "an Stilllegung gehindert" in state:
        return "Gesetzlich an Stilllegung gehindert"
    else:
        return state

In [243]:
def extract_state2(state):
    
    states = ['in Betrieb', 'Kapazitätsreserve', 'Netzreserve', 'Strommarktrückkehr', 'vorläufig stillgelegt', 'stillgelegt §13b EnWG', 'stillgelegt KVBG', 'stillgelegt']
    matches = ['In Betrieb', 'Kapazitätsreserve aufgrund von § 13e EnWG', 'Netzreserve aufgrund § 13b EnWG', 'befristete Strommarktrückkehr', 'vorläufig stillgelegt', 'endgültig stillgelegt 20[0-9]{2} \(nach § 13b EnWG\)', 'endgültig stillgelegt 20[0-9]{2} \(nach KVBG\)', '[E|e]ndgültig [S|s]tillgelegt 20[0-9]{2} \([ohne]|[mit].']
    
    for s, m in list(zip(states, matches)):
        if re.search(m, state):
            return s
    
    return state

In [244]:
extract_state2(sq)

'stillgelegt'

In [245]:
pl4['company'] = pl4['company'].apply(lambda x: fix_company(x))
#pl4['endop'] = pl4['state'].apply(lambda x: extract_still(x))
pl4['state'] = pl4['state'].apply(lambda x: extract_state2(x))
pl4['chp'] = pl4['chp'].apply(lambda x: fix_kwk(x))

In [246]:
list(pl4.groupby("state").count().reset_index()['state'])

['Kapazitätsreserve',
 'Netzreserve',
 'Strommarktrückkehr',
 'in Betrieb',
 'stillgelegt',
 'stillgelegt KVBG',
 'stillgelegt §13b EnWG',
 'vorläufig stillgelegt']

In [247]:
atest = pl4.groupby("state").count()

In [248]:
def myf():
    return 0 if "test" == "test" else 1

In [249]:
atest

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,energysource,chp,power,tech
state,,,,,,,,,,,,,,,
Kapazitätsreserve,15,15,15,15,15,15,15,14,15,15,0,15,15,15,15
Netzreserve,20,20,20,20,20,20,20,20,20,20,0,20,20,20,20
Strommarktrückkehr,26,26,26,26,26,26,26,20,26,26,0,26,26,26,26
in Betrieb,1194,944,1194,1194,1194,1194,1136,1111,1194,1194,0,1194,1194,1194,1193
stillgelegt,148,143,0,145,148,148,84,73,148,147,148,148,147,148,34
stillgelegt KVBG,15,15,0,15,14,15,13,12,15,15,15,15,15,15,14
stillgelegt §13b EnWG,9,9,0,9,9,9,9,9,9,9,9,9,9,9,9
vorläufig stillgelegt,15,13,15,15,15,15,15,14,15,15,0,15,15,15,15


In [250]:
newdf = pl4.copy()

In [251]:
#newdf.dtypes

In [252]:
newdf["state"] = newdf["state"].astype('category')
newdf["state"] = newdf["state"].cat.set_categories(STATES, ordered=True)

In [253]:
newdf["chp"] = newdf["chp"].astype('category')
newdf["chp"] = newdf["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)

In [254]:
#list(atest.reset_index()['state'])

In [255]:
#newdf.groupby("energysource").count()

In [256]:
atest

,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,energysource,chp,power,tech
state,,,,,,,,,,,,,,,
Kapazitätsreserve,15,15,15,15,15,15,15,14,15,15,0,15,15,15,15
Netzreserve,20,20,20,20,20,20,20,20,20,20,0,20,20,20,20
Strommarktrückkehr,26,26,26,26,26,26,26,20,26,26,0,26,26,26,26
in Betrieb,1194,944,1194,1194,1194,1194,1136,1111,1194,1194,0,1194,1194,1194,1193
stillgelegt,148,143,0,145,148,148,84,73,148,147,148,148,147,148,34
stillgelegt KVBG,15,15,0,15,14,15,13,12,15,15,15,15,15,15,14
stillgelegt §13b EnWG,9,9,0,9,9,9,9,9,9,9,9,9,9,9,9
vorläufig stillgelegt,15,13,15,15,15,15,15,14,15,15,0,15,15,15,15


In [257]:
pl3 = newdf

In [258]:
atest = newdf.groupby(["power"]).count()
atest.sort_values("initialop", inplace=True, ascending=False)

In [259]:
#print(atest)

In [260]:
atest = pl3.groupby(["state"]).count()
# atest

In [261]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [262]:
#bm[bm.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

In [263]:
#bm2 = bm.drop_duplicates()

In [264]:
#pl3

In [395]:
pl4 = bm.merge(pl3, how="right", left_on="sseid", right_on="blockid")

In [396]:
pl4.sort_values(by=['plantid'])

,plantid,sseid,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
1284,01-50-01000004838,BNA0269,BNA0269,BNA0269,NaN,HKW Flensburg,24939.0,Flensburg,Batteriestr.,48,Schleswig-Holstein,1970.0,2012,stillgelegt,Mineralölprodukte,Ja,30.000,NaN
1333,01-50-01000004838,BNA0274,BNA0274,BNA0274,NaN,Heizkraftwerk FL,24939.0,Flensburg,Batteriestr.,48,Schleswig-Holstein,1970.0,2016,stillgelegt,Steinkohle,Nein,23.000,NaN
1332,01-50-01000004838,BNA0273,BNA0273,BNA0273,NaN,Heizkraftwerk FL,24939.0,Flensburg,Batteriestr.,48,Schleswig-Holstein,1970.0,2016,stillgelegt,Steinkohle,Nein,35.000,NaN
287,01-50-01000004838,SEE990438768378,SEE990438768378,BNA0270,Stadtwerke Flensburg,Dampfturbine 8,24939,Flensburg,Batteriestraße,48,Schleswig-Holstein,1985.0,NaN,in Betrieb,Steinkohle,Ja,32.900,Gegendruckmaschine ohne Entnahme
288,01-50-01000004838,SEE957285549260,SEE957285549260,BNA0272,Stadtwerke Flensburg,Dampfturbine 9,24939,Flensburg,Batteriestraße,48,Schleswig-Holstein,1988.0,NaN,in Betrieb,Steinkohle,Ja,32.900,Gegendruckmaschine ohne Entnahme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,NaN,NaN,SEE997842921603,BNA1233,NaN,EVE Premnitz,14727.0,Premnitz,Dr.-Herbert-Rein-Straße,1,Brandenburg,2009.0,2021-10-16 00:00:00,stillgelegt,Abfall,Ja,25.000,Kondensationsmaschine mit Entnahme
1412,NaN,NaN,SEE940711490885,BNA1232,NaN,ZWSF Premnitz,14727.0,Premnitz,Dr.-Herbert-Rein-Straße,1,Brandenburg,2004.0,2021-06-30 00:00:00,stillgelegt,Abfall,Nein,1.648,Gegendruckmaschine ohne Entnahme
1415,NaN,NaN,SEE945819365813,BNA1055,NaN,Generator 1 EEG,65203.0,Wiesbaden,Rheingaustraße,NaN,Hessen,1981.0,2021-12-01 00:00:00,stillgelegt,Biomasse,Nein,6.720,Gegendruckmaschine ohne Entnahme
1429,NaN,NaN,SEE983424047212,NaN,NaN,HKW GT,15236.0,Frankfurt,Am Hohen Feld,4,Brandenburg,1997.0,2022-03-31 00:00:00,stillgelegt,Erdgas,Ja,24.500,Gasturbinen mit nachgeschalteter Dampfturbine


In [397]:
#pl4.sort_values(by=['power'], ascending=False)

In [398]:
#pl4.sort_values(by=['power'], ascending=False)

In [399]:
#pl4[pl4.duplicated(['BlocID'], keep=False)].sort_values("BlockID", ascending=False)

In [617]:
pl5 = pl4[pd.notnull(pl4['blockid'])]
#pl5 = pl4
pl5a = pl5[pd.notnull(pl5['power'])]

In [618]:
#pl4[pl4['plantid'].isnull()].sort_values(by=["power"])

In [619]:
#pl6['plantid'] = pl6['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [620]:
# pl7 = pl6.replace("/", "_")

In [621]:
#pl7 = pl6.apply(str.replace("/", "_"))

In [622]:
#pl6.loc[pl6.Kraftwerksname == "KW Mittelsbüren"]

In [760]:
pl6 =  pl5a.copy() #[pd.notnull(pl5a['plantid'])]

In [761]:
pl6.dtypes

plantid           object
sseid             object
blockid           object
bnaid             object
company           object
plantname         object
plz               object
place             object
street            object
streetnum         object
federalstate      object
initialop        float64
endop             object
state           category
energysource    category
chp             category
power            float64
tech              object
dtype: object

In [762]:
pl6.loc[pl6.blockid == "BNA0711"]

,plantid,sseid,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
1279,06-05-300-0326774,BNA0711,BNA0711,BNA0711,NaN,Niederaußem,50129.0,Bergheim,NaN,NaN,Nordrhein-Westfalen,1970.0,2012,stillgelegt,Braunkohle,Nein,125.0,NaN


In [763]:
def to_year2(x):
    if pd.isna(x):
        return x
    elif isinstance(x, float):
        return x
    elif isinstance(x, int):
        return x
    else:
        ts = pd.Timestamp(ts_input=x)
        #print(ts)
        return int(ts.year)

In [764]:
pl6['initialop'] = pl6['initialop'].apply(pd.to_numeric)
pl6['endop'] = pl6['endop'].apply(lambda x: to_year2(x))

In [769]:
pl6.loc[pl6.blockid == "BNA0711"].dtypes

plantid           object
sseid             object
blockid           object
bnaid             object
company           object
plantname         object
plz               object
place             object
street            object
streetnum         object
federalstate      object
initialop        float64
endop            float64
state           category
energysource    category
chp             category
power            float64
tech              object
dtype: object

In [766]:
pl6.loc[pl6.blockid == "SEE999977738880"]

,plantid,sseid,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
1413,664-02,SEE999977738880,SEE999977738880,BNA0196,NaN,IKW Deuben,6682.0,Teuchern,Industriestraße,1,Sachsen-Anhalt,1936.0,2021.0,stillgelegt KVBG,Braunkohle,Ja,67.0,Gegendruckmaschine mit Entnahme


In [770]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [771]:
#TODO: fill based on endop and state

In [772]:
blocks['state'] = blocks['state'].fillna(value='stillgelegt')

In [773]:
blocks.loc[blocks['state'].isnull()]

,plantid,sseid,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech


In [774]:
blocks.loc[blocks['state'] == 'stillgelegt'].sort_values(['endop'])

,plantid,sseid,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
1271,06-05-100-0081105,BNA0302,BNA0302,BNA0302,NaN,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,1970.0,2011.0,stillgelegt,Braunkohle,Nein,129.000,NaN
1272,06-05-100-0081105,BNA0303,BNA0303,BNA0303,NaN,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,1970.0,2011.0,stillgelegt,Braunkohle,Nein,130.000,NaN
1273,06-05-100-0081105,BNA0304,BNA0304,BNA0304,NaN,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,1970.0,2011.0,stillgelegt,Braunkohle,Nein,124.000,NaN
1274,06-05-900-0079017,BNA0414,BNA0414,BNA0414,NaN,Westfalen,59071.0,Hamm-Uentrop,NaN,NaN,Nordrhein-Westfalen,1970.0,2011.0,stillgelegt,Steinkohle,Nein,152.000,NaN
1275,06-05-900-0079017,BNA0415,BNA0415,BNA0415,NaN,Westfalen,59071.0,Hamm-Uentrop,NaN,NaN,Nordrhein-Westfalen,1970.0,2011.0,stillgelegt,Steinkohle,Nein,152.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,13-60-80407,SEE976858964187,SEE976858964187,BNA0360,NaN,HKW Gasturbine 3,17489.0,Greifswald,An der Jungfernwiese,8,Mecklenburg-Vorpommern,1996.0,2022.0,stillgelegt,Erdgas,Ja,4.894,Gasturbinen mit Abhitzekessel
1429,NaN,NaN,SEE983424047212,NaN,NaN,HKW GT,15236.0,Frankfurt,Am Hohen Feld,4,Brandenburg,1997.0,2022.0,stillgelegt,Erdgas,Ja,24.500,Gasturbinen mit nachgeschalteter Dampfturbine
1441,661-110,SEE963023008050,SEE963023008050,BNA1264,NaN,GT22,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen,1995.0,2022.0,stillgelegt,Erdgas,Ja,4.400,Gasturbinen mit Abhitzekessel
1436,NaN,NaN,SEE992475918372,NaN,NaN,BHKW Bad Am Stadtwald,24534.0,Neumünster,Hansaring,177,Schleswig-Holstein,2014.0,2022.0,stillgelegt,Mineralölprodukte,Ja,2.000,Verbrennungsmotor


In [776]:
blocks.sort_values("initialop", inplace=True, ascending=False)
# blocks

In [660]:
plants = blocks.copy()
plants = plants.dropna(subset=["plantid", "initialop"]) #TODO: remove initialop from here

In [661]:
#pl4[pl4.loc]
pl4.loc[pl4.plantid == "nan"]

,plantid,sseid,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech


In [662]:
plants.loc[plants.state == np.nan]

,plantid,sseid,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech


In [663]:
plants

,plantid,sseid,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
313,12-30532000000,SEE946069613440,SEE946069613440,NaN,Stadtwerke Frankfurt (Oder) GmbH,HKW FFO GM1,15236,Frankfurt,Am Hohen Feld,4,Brandenburg,2022.0,NaN,in Betrieb,Erdgas,Ja,10.100,Verbrennungsmotor
314,12-30532000000,SEE969511682320,SEE969511682320,NaN,Stadtwerke Frankfurt (Oder) GmbH,HKW FFO GM2,15236,Frankfurt,Am Hohen Feld,4,Brandenburg,2022.0,NaN,in Betrieb,Erdgas,Ja,10.100,Verbrennungsmotor
315,12-30532000000,SEE937997291855,SEE937997291855,NaN,Stadtwerke Frankfurt (Oder) GmbH,HKW FFO GM3,15236,Frankfurt,Am Hohen Feld,4,Brandenburg,2022.0,NaN,in Betrieb,Erdgas,Ja,10.100,Verbrennungsmotor
316,12-30532000000,SEE900549498521,SEE900549498521,NaN,Stadtwerke Frankfurt (Oder) GmbH,HKW FFO GM4,15236,Frankfurt,Am Hohen Feld,4,Brandenburg,2022.0,NaN,in Betrieb,Erdgas,Ja,10.100,Verbrennungsmotor
317,12-30532000000,SEE939848582457,SEE939848582457,NaN,Stadtwerke Frankfurt (Oder) GmbH,HKW FFO GM5,15236,Frankfurt,Am Hohen Feld,4,Brandenburg,2022.0,NaN,in Betrieb,Erdgas,Ja,10.100,Verbrennungsmotor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,06-09-676-2101-0002,SEE919296928137,SEE919296928137,"BNA0755a, BNA0755b",Kraftwerk Obernburg,Sammelschienen-Kraftwerk Gas- und Dampfturbinen,63906,Erlenbach,Glanzstoffstr.,1,Bayern,1956.0,NaN,in Betrieb,Erdgas,Ja,112.719,Gasturbinen mit nachgeschalteter Dampfturbine
687,06-05-300-9046797,SEE976489828827,SEE976489828827,BNA0600b,Currenta GmbH & Co. OHG,LEV KWG G11,51373,Leverkusen,Kaiser-Wilhelm-Allee,80,Nordrhein-Westfalen,1955.0,NaN,in Betrieb,Steinkohle,Ja,19.235,Gegendruckmaschine ohne Entnahme
55,664-07,SEE906830618081,SEE906830618081,BNA0714,RWE AG,Fab Fortuna,50129,Bergheim,Auenheimer Straße,NaN,Nordrhein-Westfalen,1948.0,NaN,in Betrieb,Braunkohle,Ja,15.000,Kondensationsmaschine mit Entnahme
1413,664-02,SEE999977738880,SEE999977738880,BNA0196,NaN,IKW Deuben,6682.0,Teuchern,Industriestraße,1,Sachsen-Anhalt,1936.0,2021.0,stillgelegt KVBG,Braunkohle,Ja,67.000,Gegendruckmaschine mit Entnahme


In [664]:
#plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [665]:
#plants_c = plants_d.groupby(["KraftwerkID"]).sum()
#plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [666]:
#plants_f = plants_c.reset_index()

In [667]:
#plants_count = plants_count.loc[:, ["blockid"]]

In [668]:
#plants_count = plants_count.reset_index()
# plants_count 

In [669]:
#plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [670]:
#plants[plants.duplicated(['KraftwerkID'], keep=False)].sort_values("BlockID", ascending=False)

In [671]:
plants["energysource"] = plants["energysource"].astype('category')
plants["energysource"] = plants["energysource"].cat.set_categories(ENERGIES, ordered=True)
plants["state"] = plants["state"].astype('category')
plants["state"] = plants["state"].cat.set_categories(STATES, ordered=True)
plants["chp"] = plants["chp"].astype('category')
plants["chp"] = plants["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants["federalstate"] = plants["federalstate"].astype('category')
plants["federalstate"] = plants["federalstate"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [672]:
pl1 = pl0.loc[pl0["energysource"].isin(["Kernenergie", "Erdgas", "Steinkohle", "Braunkohle", "Steinkohle"])]

In [673]:
p_grp = plants.groupby("plantid")

In [674]:
#plants.groupby("Energip_subgrper").max()

In [675]:
p_subgrp = plants.loc[plants['state'].isin(ACTIVE_STATES)].groupby('plantid') # TODO: add 

In [676]:
plants_act = pd.DataFrame()
for plantid, group in p_subgrp:
    entry = {}
    entry["plantid"] = plantid
    entry["activepower"] = group["power"].sum()
    plants_act = plants_act.append(entry, ignore_index=True)

/tmp/ipykernel_36473/407114243.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  plants_act = plants_act.append(entry, ignore_index=True)
/tmp/ipykernel_36473/407114243.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  plants_act = plants_act.append(entry, ignore_index=True)
/tmp/ipykernel_36473/407114243.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  plants_act = plants_act.append(entry, ignore_index=True)
/tmp/ipykernel_36473/407114243.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  plants_act = plants_act.append(entry, ignore_index=True)
/tmp/ipykernel_36473/407114243.py:6: FutureWarning: The frame.append method is depre

In [677]:
#plants_act.sort_values('activepower', ascending=False)

In [678]:
#ap = plants.loc[plants.state == "in Betrieb" | plants.state == "Gesetzlich an Stilllegung gehindert"]

In [679]:
plants_a = pd.DataFrame()
for plantid, group in p_grp:
    #if not plantid == "06-05-300-9046797":
    #    continue
    entry = {}
    #print(entry)
    entry["plantid"] = plantid
    # entry["BlockID"] = group
    try:
        entry["plantname"] = group["plantname"].value_counts().index[0]
    except IndexError:
        entry["plantname"] = np.NaN
    entry["federalstate"] = group["federalstate"].value_counts().index[0]
    entry["energysource"] = group["energysource"].min()
    try:
        entry["chp"] = group["chp"].min()
    except TypeError:
        #print("aneror")
        #print(plantid)
        #print(group['KWK'].count())
        entry["chp"] = ""
    entry["latestexpanded"] = group["initialop"].max()
    entry["initialop"] = group["initialop"].min()
    entry["totalpower"] = group["power"].sum()
    entry["state"] = group["state"].min()
    entry["blockcount"] = group["blockid"].count()
    try:
        entry["company"] = group["company"].value_counts().index[0]
    except IndexError:
        ;
    #print(entry)
    #break
    plants_a = plants_a.append(entry, ignore_index=True)

/tmp/ipykernel_36473/2910219165.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  plants_a = plants_a.append(entry, ignore_index=True)
/tmp/ipykernel_36473/2910219165.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  plants_a = plants_a.append(entry, ignore_index=True)
/tmp/ipykernel_36473/2910219165.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  plants_a = plants_a.append(entry, ignore_index=True)
/tmp/ipykernel_36473/2910219165.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  plants_a = plants_a.append(entry, ignore_index=True)
/tmp/ipykernel_36473/2910219165.py:33: FutureWarning: The frame.append method is deprecated 

In [680]:
#plants_a

In [681]:
#TODO: fix this

In [682]:
#plants_a[pd.isnull(plants_a)]

In [683]:
int_list = ['blockcount', 'initialop', 'latestexpanded']
for column in int_list:
    plants_a[column] = plants_a[column].astype(int)
#plant['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)

In [684]:
plants_a.dtypes

plantid            object
plantname          object
federalstate       object
energysource       object
chp                object
latestexpanded      int64
initialop           int64
totalpower        float64
state              object
blockcount          int64
company            object
dtype: object

In [685]:
plants_a["energysource"] = plants_a["energysource"].astype('category')
plants_a["energysource"] = plants_a["energysource"].cat.set_categories(ENERGIES, ordered=True)
plants_a["state"] = plants_a["state"].astype('category')
plants_a["state"] = plants_a["state"].cat.set_categories(STATES, ordered=True)
plants_a["chp"] = plants_a["chp"].astype('category')
plants_a["chp"] = plants_a["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants_a["federalstate"] = plants_a["federalstate"].astype('category')
plants_a["federalstate"] = plants_a["federalstate"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [686]:
plants_a.dtypes

plantid             object
plantname           object
federalstate      category
energysource      category
chp               category
latestexpanded       int64
initialop            int64
totalpower         float64
state             category
blockcount           int64
company             object
dtype: object

In [687]:
plants_a.loc[plants_a.plantid == "666-999"]

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company


In [688]:
plants_a.loc[plants_a.plantid == "06-00176010435"]

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company
35,06-00176010435,Staudinger,Hessen,Steinkohle,Ja,1992,1970,1674.0,in Betrieb,4,Uniper SE


In [689]:
plants_a.loc[plants_a.plantid == "06-05-900-0865327"]

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company
107,06-05-900-0865327,Gersteinwerk,Nordrhein-Westfalen,Steinkohle,Nein,1984,1972,2011.0,in Betrieb,9,RWE AG


In [690]:
#plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [691]:
column_titles = ['plantid', 'plantname', 'federalstate','energysource', 'chp', 'latestexpanded', 'initialop', 'totalpower', 'state', 'blockcount', 'company']
plants_b = plants_a.reindex(columns=column_titles)

In [692]:
#plants_b

In [777]:
list(blocks)

['plantid',
 'sseid',
 'blockid',
 'bnaid',
 'company',
 'plantname',
 'plz',
 'place',
 'street',
 'streetnum',
 'federalstate',
 'initialop',
 'endop',
 'state',
 'energysource',
 'chp',
 'power',
 'tech']

In [694]:
#blocks.drop(['place', 'plz', 'street'], axis=1, inplace=True)

In [778]:
blocks[blocks.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,sseid,blockid,bnaid,company,plantname,plz,place,street,streetnum,federalstate,initialop,endop,state,energysource,chp,power,tech
744,07-04-6021280,SEE966089822219,SEE966089822219,BNA1948,Kraftwerke Mainz-Wiesbaden AG,BHKW KW5 58,55120,Mainz,Kraftwerkallee,1,Rheinland-Pfalz,2020.0,NaN,in Betrieb,Erdgas,Ja,9.93,Verbrennungsmotor
743,07-04-6021280,SEE966089822219,SEE966089822219,BNA1948,Kraftwerke Mainz-Wiesbaden AG,BHKW KW5 58,55120,Mainz,Kraftwerkallee,1,Rheinland-Pfalz,2020.0,NaN,in Betrieb,Erdgas,Ja,9.93,Verbrennungsmotor


In [696]:
#blocks

In [697]:
plants_b.sort_values("initialop", ascending=False, inplace=True)

In [698]:
#drop_list = ['Blockname', 'KWK', 'Nettoleistung', 'Stilllegung', 'Straße und Hausnummer (Standort Kraftwerk)', 'PLZ', 'Ort', 'Nettoleistung']
#plants_g = plants_f.drop(drop_list, axis=1)

In [699]:
#plants_h = plants_g.rename(columns={"KraftwerkID": "plantid", "Gesamtleistung": "totalpower", "Blockzahl": "blockcount", "BlockID": "blockid", "Bundesland": "federalstate", "Energieträger": "energysource", "Inbetriebnahme": "latestexpanded", "Kraftwerksname": "plantname", "Status": "state", "Unternehmen": "company"})

In [700]:
plants_final = plants_b.loc[:]

In [701]:
plants_final.dtypes

plantid             object
plantname           object
federalstate      category
energysource      category
chp               category
latestexpanded       int64
initialop            int64
totalpower         float64
state             category
blockcount           int64
company             object
dtype: object

In [702]:
# plants_final

In [703]:
# stammdaten

In [704]:
#stammdaten

In [705]:
# cols = [0,2,3,9,10,11,12]
stammdaten = pl6.copy()
drop_list = ['plantid', 'energysource', 'initialop', 'chp', 'plantname', 'power', 'state', 'endop', 'company', 'tech', 'bnaid', 'sseid']
stammdaten_dafuq = stammdaten.drop(drop_list, axis=1)
stammdaten = stammdaten_dafuq.copy()

In [706]:
stammdaten_dafuq.sort_values(['blockid'])

,blockid,plz,place,street,streetnum,federalstate
1276,BNA0011,79774.0,Albbruck,NaN,NaN,Baden-Württemberg
1357,BNA0012d,31061.0,Alfeld,Mühlenmarsch,1,Niedersachsen
1297,BNA0059a,34225.0,Baunatal,NaN,NaN,Hessen
1374,BNA0075,12207.0,Berlin,Ostpreußendamm,61,Berlin
1358,BNA0076,12207.0,Berlin,Ostpreußendamm,61,Berlin
...,...,...,...,...,...,...
733,SEE999819576335,39126,Magdeburg,Kraftwerk-Privatweg,7,Sachsen-Anhalt
233,SEE999839188105,40476,Düsseldorf,Rather Straße,51,Nordrhein-Westfalen
1398,SEE999848168525,21079.0,Hamburg,Moorburger Schanze,2,Hamburg
955,SEE999966551940,14478,Potsdam,Zum Heizwerk,20,Brandenburg


In [707]:
stmp = pl6.copy()
stammdaten2 = stmp.merge(prtr, how="left", left_on="plantid", right_on="kennnummer")

In [708]:
#stammdaten2.loc[stammdaten2.blockid == "BNA0221c"]

In [709]:
#stammdaten.to_csv("stammdaten.csv", index=False)

In [710]:
#plants_final.to_csv("plants.csv", index=False)

In [711]:
#blocks.to_csv("blocks_3.csv", index=False)

In [712]:
#stammdaten2

In [713]:
DROP_ST = ['blockid', 'jahr', 'kennnummer','betriebsname','betriebsname_2','plz_x','ort','strasse','hausnr','bundesland','flusseinzugsgebiet','taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']
DROP_ST2 = ['jahr', 'kennnummer','betriebsname','betriebsname_2','plz','ort','strasse','hausnr','bundesland','flusseinzugsgebiet','taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']
DROP_ST3 = ['jahr', 'kennnummer', 'bundesland', 'flusseinzugsgebiet', "betriebsname", "betriebsname_2", 'taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']


In [714]:
#DROP_ST = ['jahr']

In [715]:
plants_d1 = plants_final.merge(prtr, how="left", left_on="plantid", right_on="kennnummer")

In [716]:
#plants_d1.dtypes

In [717]:
pld2 = plants_d1.drop_duplicates(['plantid'])

In [718]:
pld3 = pld2.drop(DROP_ST3, axis=1)
plants_final1 = pld3

In [719]:
plants_final2 = plants_final1.merge(plants_act, on="plantid", how='left')

In [720]:
plants_final3 = plants_final2.dropna(subset=["plantid"])

In [721]:
plants_final3.groupby("energysource").count()

,plantid,plantname,federalstate,chp,latestexpanded,initialop,totalpower,state,blockcount,company,plz,ort,strasse,hausnr,geo_lat_wgs84,geo_long_wgs84,activepower
energysource,,,,,,,,,,,,,,,,,
Kernenergie,8,8,8,8,8,8,8,8,8,3,0,0,0,0,0,0,3
Braunkohle,30,30,30,30,30,30,30,30,30,25,28,28,28,28,28,28,25
Steinkohle,70,70,70,70,70,70,70,70,70,53,66,66,66,66,66,66,53
Erdgas,195,193,195,195,195,195,195,195,195,176,125,125,125,125,125,125,176
Mineralölprodukte,20,20,20,20,20,20,20,20,20,15,7,7,7,7,7,7,15
Abfall,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Biomasse,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [722]:
plants_final2

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company,plz,ort,strasse,hausnr,geo_lat_wgs84,geo_long_wgs84,activepower
0,06-11-01-1105502,GuD Marzahn Dampfturbine,Berlin,Erdgas,Ja,2020,2020,258.800,in Betrieb,2,Vattenfall GmbH,12681,Berlin,Rhinstr.,70,52.523389,13.522089,258.800
1,06-09-162-0025-0001,Werk 1.1 KWK 1,Bayern,Erdgas,Ja,2019,2019,26.940,in Betrieb,7,BMW AG,80788,München,Lerchenauer Str.,76,48.179175,11.561502,26.940
2,661-101,Heizkraftwerk Stendal,Sachsen-Anhalt,Erdgas,Ja,2018,2018,13.200,in Betrieb,1,"Stadtwerke - Altmärkische Gas-, Wasser-und Ele...",NaN,NaN,NaN,NaN,NaN,NaN,13.200
3,661-93,HKW Dresden-Nord,Sachsen,Erdgas,Ja,2018,2018,11.500,in Betrieb,1,DREWAG - Stadtwerke Dresden GmbH,NaN,NaN,NaN,NaN,NaN,NaN,11.500
4,661-111,Gasmotor Bad Salzungen,Thüringen,Erdgas,Ja,2018,2018,10.003,in Betrieb,1,TEAG Thüringer Energie AG,NaN,NaN,NaN,NaN,NaN,NaN,10.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,06-09-676-2101-0002,Sammelschienen-Kraftwerk Gas- und Dampfturbinen,Bayern,Erdgas,Ja,1956,1956,112.719,in Betrieb,1,Kraftwerk Obernburg,63906,Erlenbach a.Main,Glanzstoffstr.,1,49.827408,9.148631,112.719
320,06-05-300-9046797,LEV KWG G23,Nordrhein-Westfalen,Steinkohle,Ja,1999,1955,128.000,in Betrieb,8,Currenta GmbH & Co. OHG,51368,Leverkusen,CHEMPARK Leverkusen,Gebäude D 8,51.022209,6.985901,128.000
321,664-07,Fab Fortuna,Nordrhein-Westfalen,Braunkohle,Ja,1948,1948,15.000,in Betrieb,1,RWE AG,NaN,NaN,NaN,NaN,NaN,NaN,15.000
322,664-02,IKW Deuben,Sachsen-Anhalt,Braunkohle,Ja,1936,1936,67.000,stillgelegt KVBG,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [723]:
plants_final2[pd.isnull(plants_final2['plantid'])]

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company,plz,ort,strasse,hausnr,geo_lat_wgs84,geo_long_wgs84,activepower


In [724]:
stammdaten

,blockid,plz,place,street,streetnum,federalstate
0,SEE946736241334,73432,Aalen,Palm Allee,1,Baden-Württemberg
1,SEE930596800480,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg
2,SEE929797382345,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg
3,SEE988046628214,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg
4,SEE923527691592,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg
...,...,...,...,...,...,...
1438,SEE963177319026,75175.0,Pforzheim,Hohwiesenweg,15,Baden-Württemberg
1439,SEE974753539035,75175.0,Pforzheim,Hohwiesenweg,15,Baden-Württemberg
1440,SEE943101874623,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen
1441,SEE963023008050,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen


In [725]:
#plants_final2.sort_values("activepower", ascending=False)

In [726]:
#stammdaten2.dtypes

In [727]:
#stammdaten3.sort_values(by=['plantid'], ascending=True)

In [728]:
#stammdaten3 = stammdaten2.drop(DROP_ST, axis=1)

In [729]:
#stammdaten4 = stammdaten3.drop_duplicates(['bnaid'])

In [730]:
#stammdaten4

In [731]:
#list(stammdaten2)

In [732]:
'''


"CREATE TABLE blocks(plantid TEXT, blockid TEXT NOT NULL PRIMARY KEY, blockdescription TEXT, federalstate TEXT, energysource TEXT, initialop INTEGER, chp TEXT,
blockname TEXT, netpower REAL, state TEXT, endop TEXT, company TEXT, FOREIGN KEY (blockid) REFERENCES addresses(blockid) ON DELETE CASCADE);"
662-01|BNA0164|Vattenfall GmbH|Brunsbüttel|Schleswig-Holstein|GT D|0.0|stillgelegt|Mineralölprodukte|Nein|63.5|201

'''


'\n\n\n"CREATE TABLE blocks(plantid TEXT, blockid TEXT NOT NULL PRIMARY KEY, blockdescription TEXT, federalstate TEXT, energysource TEXT, initialop INTEGER, chp TEXT,\nblockname TEXT, netpower REAL, state TEXT, endop TEXT, company TEXT, FOREIGN KEY (blockid) REFERENCES addresses(blockid) ON DELETE CASCADE);"\n662-01|BNA0164|Vattenfall GmbH|Brunsbüttel|Schleswig-Holstein|GT D|0.0|stillgelegt|Mineralölprodukte|Nein|63.5|201\n\n'

In [779]:
#|664-02|IKW

In [780]:
blocks2 = blocks[['blockid', 'plantid', 'plantname', 'federalstate', 'energysource', 'initialop', 'chp', 'power', 'state', 'endop', 'company']]

In [781]:
blocks2.loc[blocks2.blockid == "SEE999977738880"]

,blockid,plantid,plantname,federalstate,energysource,initialop,chp,power,state,endop,company
1413,SEE999977738880,664-02,IKW Deuben,Sachsen-Anhalt,Braunkohle,1936.0,Ja,67.0,stillgelegt KVBG,2021.0,NaN


In [782]:
blocks2.loc[blocks2.blockid == "SEE982928435068"]

,blockid,plantid,plantname,federalstate,energysource,initialop,chp,power,state,endop,company
74,SEE982928435068,06-11-01-1105502,GuD Marzahn Dampfturbine,Berlin,Erdgas,2020.0,Ja,69.3,in Betrieb,NaN,Vattenfall GmbH


In [ ]:
list(blocks2)

In [ ]:
#blocks2.loc['C', 'x'] = "BNA1949"

In [783]:
stammdaten.to_csv("stammdaten_nh_new.csv", index=False, header=False)
blocks2.to_csv("blocks_nh.csv", index=False, header=False)
plants_final2.to_csv("plants_nh.csv", index=False, header=False)
stammdaten.to_csv("stammdaten.csv", index=False)
blocks2.to_csv("blocks.csv", index=False)
plants_final2.to_csv("plants.csv", index=False)

In [ ]:
#sqlite3 plantwatch.db  "CREATE TABLE plants(plantid TEXT NOT NULL PRIMARY KEY, plantname TEXT, federalstate TEXT, energysource TEXT, chp TEXT, latestexpanded INT, initialop INT, totalpower REAL, state TEXT, blockcount INT,  company TEXT, plz TEXT, place TEXT, street TEXT, number TEXT, latitude REAL, longitude REAL, activepower REAL, energy_2015 INTEGER,  energy_2016 INTEGER,  energy_2017 INTEGER,  energy_2018 INTEGER,  energy_2019 INTEGER, energy_2020 INTEGER, energy_2021 INTEGER, co2_2007 INTEGER,  co2_2008 INTEGER,  co2_2009 INTEGER,  co2_2010 INTEGER,  co2_2011 INTEGER,  co2_2012 INTEGER,  co2_2013 INTEGER,  co2_2014 INTEGER,  co2_2015 INTEGER,  co2_2016 INTEGER,  co2_2017 INTEGER,  co2_2018 INTEGER, co2_2019 INTEGER, co2_2020 INTEGER, FOREIGN KEY (plantid) REFERENCES blocks(plantid) ON DELETE CASCADE);"


In [ ]:
blocks2.dropna(subset=['plantid'])

In [ ]:
blocks2.shape

In [ ]:
blocks2.dropna(subset=['plantid']).shape

In [ ]:
stammdaten.dropna(subset=['sseid', 'bnaid']).sort_values(by=['sseid'], ascending=False)

In [ ]:
stammdaten

In [ ]:
blocks.loc[blocks["blockid"] == "BNA0645"]

In [ ]:
#bm2.to_csv("bpm.csv", index=False)

In [ ]:
#blocks

In [ ]:
#blocks.groupby('Energieträger').count()

In [ ]:
#pl4 = blocks.loc[blocks['Energieträger'] == "Mineralölprodukte"]

In [ ]:
#pl4

In [ ]:
#pd.set_option('display.max_rows', None)

In [ ]:
#pl4.loc[pl4['Energieträger'] == "Mineralölprodukte"].sort_values(["Bundesland", "Unternehmen", "Kraftwerksname"])

In [ ]:
#plants_final2

In [ ]:
blocks2.sort_values('power', ascending=False)[0:20]

In [ ]:
blocks2.sort_values('power', ascending=False)[0:20]